In [1]:
import pandas as pd
import numpy as np

import sys
sys.path.append('../')
from utils.pickle_helper import *
from utils.csv_helper import *
from utils.onehot_helper import *

In [2]:
# one-hot 编码
# onehot('test_a.bin', 'test_a')
# onehot('test_b.bin', 'test_b')

In [3]:
#t load dataframe, models
pddf = read_pickle('test_b.dataframe.bin')
model_svr_rbf = read_pickle('svr_rbf.model.bin')
model_xgbr = read_pickle('xgbr.model.bin')
model_xgbr_gs = read_pickle('xgbr2.model.bin')
remain_header = read_pickle('remain_header.bin')
est = read_pickle('pca.bin')
ymin, ymax = read_pickle('ys.bin')
pddf.info()

D:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


<class 'pandas.core.frame.DataFrame'>
Index: 120 entries, ID856 to ID995
Columns: 8076 entries, TOOL_ID_0 to 750X1452
dtypes: float64(6034), int64(1980), object(62)
memory usage: 7.4+ MB


In [4]:
# sample index
header = list(pddf.index)
print(header)

['ID856', 'ID857', 'ID858', 'ID859', 'ID860', 'ID861', 'ID862', 'ID863', 'ID864', 'ID865', 'ID867', 'ID869', 'ID870', 'ID871', 'ID872', 'ID873', 'ID874', 'ID875', 'ID876', 'ID877', 'ID878', 'ID879', 'ID880', 'ID881', 'ID882', 'ID883', 'ID884', 'ID885', 'ID886', 'ID887', 'ID888', 'ID889', 'ID890', 'ID891', 'ID892', 'ID893', 'ID894', 'ID895', 'ID896', 'ID897', 'ID900', 'ID901', 'ID902', 'ID903', 'ID904', 'ID905', 'ID906', 'ID907', 'ID908', 'ID909', 'ID910', 'ID911', 'ID912', 'ID915', 'ID916', 'ID917', 'ID918', 'ID919', 'ID920', 'ID923', 'ID924', 'ID925', 'ID926', 'ID927', 'ID928', 'ID929', 'ID930', 'ID932', 'ID934', 'ID936', 'ID938', 'ID939', 'ID940', 'ID941', 'ID942', 'ID943', 'ID944', 'ID945', 'ID946', 'ID947', 'ID948', 'ID949', 'ID950', 'ID951', 'ID952', 'ID953', 'ID954', 'ID955', 'ID956', 'ID957', 'ID958', 'ID959', 'ID960', 'ID961', 'ID962', 'ID963', 'ID964', 'ID965', 'ID966', 'ID967', 'ID968', 'ID969', 'ID970', 'ID972', 'ID973', 'ID975', 'ID977', 'ID978', 'ID979', 'ID982', 'ID984', 

In [5]:
x = np.array(pddf[remain_header])
np.shape(x)

(120, 185)

In [6]:
# 缺失值填充: median 表示中位数, mean 表示平均数
from sklearn.preprocessing import Imputer
# imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
imp = Imputer(missing_values='NaN', strategy='median', axis=0)
x_imp = imp.fit_transform(x)
# x_iimp = imp.inverse_transform(x_imp)
np.shape(x_imp)

(120, 185)

In [7]:
# 数据标准化: 最大最小值
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x_scl = scaler.fit_transform(x_imp)
np.shape(x_scl)

(120, 185)

In [8]:
x_pca = est.transform(x_scl)
np.shape(x_pca)

(120, 37)

In [9]:
def gy(u): # g: u -> y
    return u
#     a = 0.172020013840487
#     b = 2.355955139758581
#     c = 0.028332501141596
#     d = 0.450477070771684
#     return a * np.tan(b * u + c) + d

def rebuild(a, amax, amin):
    return a * (amax - amin) + amin;

In [10]:
# svm
y_pred = model_svr_rbf.predict(x_pca)
yh_pred = gy(y_pred)
yh_rebuild = rebuild(yh_pred, ymin, ymax)
np.shape(yh_rebuild)

(120,)

In [11]:
# # xgbr
# y_pred2 = model_xgbr.predict(x_pca)
# yh_pred2 = gy(y_pred2)
# yh_rebuild2 = rebuild(yh_pred2, ymin, ymax)
# np.shape(yh_rebuild2)

In [12]:
# xgbr2
y_pred3 = model_xgbr_gs.predict(x_pca)
yh_pred3 = gy(y_pred3)
yh_rebuild3 = rebuild(yh_pred3, ymin, ymax)
np.shape(yh_rebuild3)

(120,)

In [13]:
mse, mae = 0, 0
yh_rebuild2 = yh_rebuild3;
for i in range(120):
    a = yh_rebuild[i]
    b = yh_rebuild3[i]
    yh_rebuild2[i] = (a + b) * 0.5
    err = np.abs(a - b)
    err2 = err * err
    mae += err
    mse += err2
    print('%.7f\t%.7f\t%.7f\t%.7f' % (a, b, err, err2))
print('MAE = %f\nMSE = %f' % (mae / 120.0, mse / 120.0))

2.9068890	2.9091613	0.0022724	0.0000052
2.8255882	2.8401799	0.0145918	0.0002129
3.0977801	3.0176992	0.0800808	0.0064129
3.0071630	3.0579312	0.0507682	0.0025774
2.9345317	2.9324756	0.0020562	0.0000042
2.7724810	2.9001284	0.1276474	0.0162939
3.0070155	2.9164445	0.0905710	0.0082031
3.1088611	3.0077825	0.1010787	0.0102169
2.9120758	2.9948232	0.0827474	0.0068471
2.8808944	2.9259343	0.0450399	0.0020286
2.7846000	2.8492050	0.0646050	0.0041738
2.8764780	2.8514667	0.0250114	0.0006256
2.8339382	2.8828621	0.0489239	0.0023935
2.8760236	2.9734232	0.0973997	0.0094867
2.9971551	2.9305058	0.0666493	0.0044421
2.8352511	2.9177523	0.0825012	0.0068064
3.1293501	3.0292013	0.1001488	0.0100298
3.1555112	3.0391152	0.1163960	0.0135480
2.9957402	2.9418375	0.0539026	0.0029055
3.1019002	3.0264542	0.0754460	0.0056921
2.9601624	2.9790828	0.0189204	0.0003580
2.9168658	2.8285146	0.0883512	0.0078059
2.9718216	2.9370239	0.0347978	0.0012109
3.0331661	2.9302099	0.1029562	0.0106000
3.1528221	2.9755182	0.1773039	0.0314367


In [14]:
# mse, mae = 0, 0
# for i in range(100):
#     a = pddf['Y'][i]
#     b = yh_rebuild3[i]
#     err = np.abs(a - b)
#     err2 = err * err
#     mae += err
#     mse += err2
#     print('%.7f\t%.7f\t%.7f\t%.7f' % (a, b, err, err2))
# print('MAE = %f\nMSE = %f' % (mae / 120.0, mse / 120.0))

In [15]:
data = []
# for i in range(100):
#     data.append([header[i], (yh_rebuild[i] + yh_rebuild2[i]) / 2.0])
# write_csv('ans_test_a_svr_rbf_xgbr.csv', data)
for i in range(100):
    data.append([header[i], (yh_rebuild2[i])])
write_csv('ans_test_b_xgbr_gs.csv', data)